In [1]:
import pandas as pd

In [ ]:
# Logic : All files under app_running follows similar structure.
# Strategy : Reading a sample file and using it to generalize the rest 

df_00 = pd.read_csv('dataset/call_log/call_log_u00.csv')
df_00.head()

,id,device,timestamp,CALLS__id,CALLS_date,CALLS_duration,CALLS_name,CALLS_number,CALLS_numberlabel,CALLS_numbertype,CALLS_type
0,8b67cbb5-cfa2-465b-b5af-31ee2c6606a6-40,1977b545-a88f-4903-a7ae-2c434de4be49,1364099483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9c0354e8-4f4f-451a-9358-efe17df0d26c-22,1977b545-a88f-4903-a7ae-2c434de4be49,1364077455,213.0,1.364077e+12,10.0,"{""ONE_WAY_HASH"":""70f8bb9a8a5393ef080507a89e4b9...","{""ONE_WAY_HASH"":""c51aad31359925622a961f2082ecf...",NaN,"{""ONE_WAY_HASH"":""356a192b7913b04c54574d18c28d4...",2.0
2,9c0354e8-4f4f-451a-9358-efe17df0d26c-22,1977b545-a88f-4903-a7ae-2c434de4be49,1364077455,212.0,1.364077e+12,13.0,"{""ONE_WAY_HASH"":""70f8bb9a8a5393ef080507a89e4b9...","{""ONE_WAY_HASH"":""c51aad31359925622a961f2082ecf...",NaN,"{""ONE_WAY_HASH"":""356a192b7913b04c54574d18c28d4...",2.0
3,9c0354e8-4f4f-451a-9358-efe17df0d26c-22,1977b545-a88f-4903-a7ae-2c434de4be49,1364077455,211.0,1.364075e+12,18.0,"{""ONE_WAY_HASH"":""70f8bb9a8a5393ef080507a89e4b9...","{""ONE_WAY_HASH"":""c51aad31359925622a961f2082ecf...",NaN,"{""ONE_WAY_HASH"":""356a192b7913b04c54574d18c28d4...",2.0
4,9c0354e8-4f4f-451a-9358-efe17df0d26c-22,1977b545-a88f-4903-a7ae-2c434de4be49,1364077455,210.0,1.364074e+12,438.0,"{""ONE_WAY_HASH"":""70f8bb9a8a5393ef080507a89e4b9...","{""ONE_WAY_HASH"":""c51aad31359925622a961f2082ecf...",NaN,"{""ONE_WAY_HASH"":""356a192b7913b04c54574d18c28d4...",2.0


In [3]:
df_00.nunique()

id                   1759
device                  1
timestamp            1757
CALLS__id             629
CALLS_date            629
CALLS_duration        195
CALLS_name             13
CALLS_number           71
CALLS_numberlabel       0
CALLS_numbertype        3
CALLS_type              3
dtype: int64

In [ ]:
df_00.shape

(2203, 11)

In [5]:
df_00.columns

Index(['id', 'device', 'timestamp', 'CALLS__id', 'CALLS_date',
       'CALLS_duration', 'CALLS_name', 'CALLS_number', 'CALLS_numberlabel',
       'CALLS_numbertype', 'CALLS_type'],
      dtype='object')

In [6]:
cols_to_drop = ['id','CALLS_name','CALLS_number','CALLS_numberlabel','CALLS_numbertype']
cleaned_df = df_00.drop(columns=cols_to_drop)

In [7]:
cleaned_df.loc[:,'date-time'] = pd.to_datetime(cleaned_df['timestamp'],unit='s')
cleaned_df.loc[:,'date'] = cleaned_df['date-time'].dt.date

In [ ]:
# When call id is missing , remaining data is also missing
cleaned_df[cleaned_df.CALLS__id.isna()].isna().sum()

device               0
timestamp            0
CALLS__id         1566
CALLS_date        1566
CALLS_duration    1566
CALLS_type        1566
date-time            0
date                 0
dtype: int64

In [56]:
cleaned_df = cleaned_df[~cleaned_df.CALLS__id.isna()]

In [ ]:
final_df = cleaned_df.groupby('date').agg(
    device = ('device','max'),
    total_calls = ('timestamp','count'),
    total_duration = ('CALLS_duration','sum'),
    longest_call = ('CALLS_duration','max'),
    average_call_time = ('CALLS_duration','mean'),
    frequent_call_type = ('CALLS_type',
                          lambda x: x.value_counts().idxmax() # incoming / outgoing / missed
                        )
)

In [75]:
final_df

,device,total_calls,total_duration,longest_call,average_call_time,frequent_call_type
date,,,,,,
2013-03-23,1977b545-a88f-4903-a7ae-2c434de4be49,213,22394.0,3320.0,105.136150,2.0
2013-03-24,1977b545-a88f-4903-a7ae-2c434de4be49,2,39.0,20.0,19.500000,1.0
2013-03-25,1977b545-a88f-4903-a7ae-2c434de4be49,2,512.0,421.0,256.000000,2.0
2013-03-26,1977b545-a88f-4903-a7ae-2c434de4be49,6,227.0,159.0,37.833333,2.0
2013-03-27,1977b545-a88f-4903-a7ae-2c434de4be49,7,1665.0,1002.0,237.857143,3.0
...,...,...,...,...,...,...
2013-06-11,1977b545-a88f-4903-a7ae-2c434de4be49,7,264.0,111.0,37.714286,2.0
2013-06-12,1977b545-a88f-4903-a7ae-2c434de4be49,9,617.0,355.0,68.555556,2.0
2013-06-13,1977b545-a88f-4903-a7ae-2c434de4be49,2,60.0,46.0,30.000000,2.0


In [8]:
# Returning Data Frame , Some containing Call Duration Data, while some not (based on the availability of the data).  
# Call duration is not droped as it can be used for furthur analysis
def get_call_log_df(filename):
    
    df_00 = pd.read_csv(filename)
    
    df_00.loc[:,'datetime'] = pd.to_datetime(df_00['timestamp'],unit='s')
    df_00.loc[:,'date'] = df_00['datetime'].dt.date
    
    call_count_df = df_00.groupby('date').agg(
        device = ('device','max'),
        total_calls = ('timestamp','count'),
    )
    
    if 'CALLS_duration' in df_00.columns:
        
        cols_to_drop = [
            'id',
            'CALLS_name',
            'CALLS_number',
            'CALLS_numberlabel',
            'CALLS_numbertype'
        ]
        cleaned_df = df_00.drop(columns=cols_to_drop)
        
        cleaned_df = df_00[~df_00.CALLS__id.isna()]
        
        call_duration_df = cleaned_df.groupby('date').agg(
            total_duration_s = ('CALLS_duration','sum'),
                longest_call_s = ('CALLS_duration','max'),
                average_call_time_s = ('CALLS_duration','mean'),
                frequent_call_type = ('CALLS_type',
                                    lambda x: x.value_counts().idxmax() # incoming / outgoing / missed
                                    )
        )
        final_df = pd.merge(call_count_df,call_duration_df,on='date',how='left')
        final_df['uid'] = filename.split('_')[-1].replace('.csv','')
        
        return final_df
    #else
    
    call_count_df['uid'] = filename.split('_')[-1].replace('.csv','')
    return call_count_df

In [9]:
df = get_call_log_df('../dataset/call_log/call_log_u00.csv')
df

,device,total_calls,total_duration_s,longest_call_s,average_call_time_s,frequent_call_type,uid
date,,,,,,,
2013-03-23,1977b545-a88f-4903-a7ae-2c434de4be49,213,22394.0,3320.0,105.136150,2.0,u00
2013-03-24,1977b545-a88f-4903-a7ae-2c434de4be49,21,39.0,20.0,19.500000,1.0,u00
2013-03-25,1977b545-a88f-4903-a7ae-2c434de4be49,23,512.0,421.0,256.000000,2.0,u00
2013-03-26,1977b545-a88f-4903-a7ae-2c434de4be49,26,227.0,159.0,37.833333,2.0,u00
2013-03-27,1977b545-a88f-4903-a7ae-2c434de4be49,27,1665.0,1002.0,237.857143,3.0,u00
...,...,...,...,...,...,...,...
2013-06-11,1977b545-a88f-4903-a7ae-2c434de4be49,29,264.0,111.0,37.714286,2.0,u00
2013-06-12,1977b545-a88f-4903-a7ae-2c434de4be49,32,617.0,355.0,68.555556,2.0,u00
2013-06-13,1977b545-a88f-4903-a7ae-2c434de4be49,7,60.0,46.0,30.000000,2.0,u00


In [ ]:
# Merging all student datas
import os
root_dir = '../dataset/call_log'
# os.path.join('dataset','app_usage')
user_files = os.listdir(path=root_dir)
combined_dfs = []
for file in user_files:
    print('Ingesting : ',file)
    file_path = os.path.join(root_dir,file)
    df= get_call_log_df(file_path)
    combined_dfs.append(df)
call_log_df = pd.concat(combined_dfs)

Ingesting :  call_log_u13.csv
Ingesting :  call_log_u32.csv
Ingesting :  call_log_u24.csv
Ingesting :  call_log_u18.csv
Ingesting :  call_log_u34.csv
Ingesting :  call_log_u45.csv
Ingesting :  call_log_u51.csv
Ingesting :  call_log_u27.csv
Ingesting :  call_log_u30.csv
Ingesting :  call_log_u17.csv
Ingesting :  call_log_u02.csv
Ingesting :  call_log_u44.csv
Ingesting :  call_log_u49.csv
Ingesting :  call_log_u36.csv
Ingesting :  call_log_u20.csv
Ingesting :  call_log_u08.csv
Ingesting :  call_log_u58.csv
Ingesting :  call_log_u35.csv
Ingesting :  call_log_u15.csv
Ingesting :  call_log_u59.csv
Ingesting :  call_log_u01.csv
Ingesting :  call_log_u50.csv
Ingesting :  call_log_u00.csv
Ingesting :  call_log_u05.csv
Ingesting :  call_log_u25.csv
Ingesting :  call_log_u53.csv
Ingesting :  call_log_u42.csv
Ingesting :  call_log_u22.csv
Ingesting :  call_log_u23.csv
Ingesting :  call_log_u41.csv
Ingesting :  call_log_u54.csv
Ingesting :  call_log_u16.csv
Ingesting :  call_log_u56.csv
Ingesting 

In [42]:
# Percentage of data missing
call_log_missing = call_log_df.isna().sum()/call_log_df.shape[0]*100
call_log_missing

device                  0.000000
total_calls             0.000000
total_duration_s       78.204294
longest_call_s         78.204294
average_call_time_s    78.204294
frequent_call_type     78.204294
uid                     0.000000
dtype: float64

In [ ]:
students_call_duration_missing = call_log_df.groupby('uid').agg(
    missing_call_duration = ('total_duration_s',lambda x: x.isna().sum()/len(x)*100)
)
students_call_duration_missing.sort_values(by ='missing_call_duration',ascending=False)

# most student doesnt have their call recordings recorded.

,missing_call_duration
uid,
u01,100.000000
u17,100.000000
u03,100.000000
u04,100.000000
u05,100.000000
u09,100.000000
u07,100.000000
u14,100.000000
u10,100.000000


In [ ]:
# removing columns where more than 50% of the data is missing
cols_to_drop = call_log_missing[call_log_missing>50].index
call_log_df = call_log_df.drop(columns=cols_to_drop)
call_log_df

,device,total_calls,uid
date,,,
2013-03-29,83123b91-10e9-4bd4-8214-42d04e10dc14,22,u13
2013-03-30,83123b91-10e9-4bd4-8214-42d04e10dc14,24,u13
2013-03-31,83123b91-10e9-4bd4-8214-42d04e10dc14,22,u13
2013-04-01,83123b91-10e9-4bd4-8214-42d04e10dc14,24,u13
2013-04-02,83123b91-10e9-4bd4-8214-42d04e10dc14,76,u13
...,...,...,...
2013-05-22,25fd565d-66d3-429f-a733-58bcd9254859,23,u31
2013-05-23,25fd565d-66d3-429f-a733-58bcd9254859,21,u31
2013-05-24,55a4168c-5b87-4d6b-81a2-82b17666aaf1,124,u31
